In [4]:
# 머신러닝 이용하기

# XOR 계산 결과 데이터
from sklearn import svm
xor_data = [
    [0, 0, 0],
    [0, 1, 1],
    [1, 0, 1],
    [1, 1, 0]
]

# 학습을 위해 데이터와 레이블 분리하기
data = []
label = []
for row in xor_data:
    p = row[0]
    q = row[1]
    r = row[2]
    data.append([p,q])
    label.append(r)

# 데이터 학습시키기
clf = svm.SVC()
clf.fit(data, label)

# 데이터 예측하기
pre = clf.predict(data)
print("예측결과: ", pre)

# 결과 확인하기
ok = 0; total = 0
for idx, answer in enumerate(label):
    p = pre[idx]
    if p == answer: ok += 1
    total += 1
print("정답률: ", ok, "/", total, "=", ok/total)

예측결과:  [0 1 1 0]
정답률:  4 / 4 = 1.0


In [8]:
# 프레임워크로 프로그램 간단하게 작성하기

import pandas as pd
from sklearn import svm, metrics

# XOR 연산
xor_input = [
    [0, 0, 0],
    [0, 1, 1],
    [1, 0, 1],
    [1, 1, 0]
]

# 입력을 학습 전용 데이터와 테스트 전용 데이터로 분류하기
xor_df = pd.DataFrame(xor_input)
xor_data = xor_df.ix[:,0:1]
xor_label = xor_df.ix[:,2]

# 데이터 학습과 예측하기
clf = svm.SVC()
clf.fit(xor_data, xor_label)
pre = clf.predict(xor_data)

# 정답률 구하기
ac_score = metrics.accuracy_score(xor_label, pre) # 함수 이용하면 정답률을 쉼게 구할 수 있다.
print("정답률 = " , ac_score)

정답률 =  1.0


C:\Program Files\Anaconda3\lib\site-packages\ipykernel\__main__.py:11: DeprecationWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/indexing.html#ix-indexer-is-deprecated


In [11]:
# 머신러닝으로 붓꽃 품종 분류하기

from sklearn import svm, metrics
import random, re

csv =[]
with open('iris.csv', 'r', encoding = 'utf-8') as fp:
    for line in fp:
        line = line.strip()
        cols = line.split(',')
        fn = lambda n : float(n) if re.match(r'^[0-9\.]+$', n) else n
        cols = list(map(fn, cols))
        csv.append(cols)
        
del csv[0]

random.shuffle(csv)

total_len = len(csv)
train_len = int(total_len * 2 / 3)
train_data = []
train_label = []
test_data = []
test_label = []

for i in range(total_len):
    data = csv[i][0:4]
    label = csv[i][4]
    if i < train_len:
        train_data.append(data)
        train_label.append(label)
    else:
        test_data.append(data)
        test_label.append(label)
        
clf = svm.SVC()
clf.fit(train_data, train_label)
pre = clf.predict(test_data)

ac_score = metrics.accuracy_score(test_label, pre)
print("정답률 = ", ac_score)

정답률 =  0.98


In [14]:
# 훈련 전용 데이터와 테스트 전용 데이터로 분할하는 메서드

import pandas as pd
from sklearn import svm, metrics
from sklearn.model_selection import train_test_split
# 붓꽃의 csv 데이터 읽어 들이기
csv = pd.read_csv('iris.csv')
# 필요한 열 추출하기
csv_data = csv[["SepalLength", "SepalWidth", "PetalLength", "PetalWidth"]]
csv_label = csv["Name"]
# 학습 전용 데이터와 테스트 전용 데이터로 나누기
train_data, test_data, train_label, test_label = train_test_split(csv_data, csv_label)
# 데이터 학습시키고 예측하기
clf = svm.SVC()
clf.fit(train_data, train_label)
pre = clf.predict(test_data)
# 정답률 구하기
ac_score = metrics.accuracy_score(test_label, pre)
print("정답률 = ", ac_score)

정답률 =  0.9736842105263158


In [20]:
# 이미지 내부의 문자인식
# GZip 압축을 해제하는 프로그램

import urllib.request as req
import gzip, os, os.path

savepath = "./mnist"
baseurl = "http://yann.lecun.com/exdb/mnist"
files = [
    "train-images-idx3-ubyte.gz",
    "train-labels-idx1-ubyte.gz",
    "t10k-images-idx3-ubyte.gz",
    "t10k-labels-idx1-ubyte.gz"]

# 다운로드
if not os.path.exists(savepath): os.mkdir(savepath)
for f in files:
    url = baseurl + "/" + f
    loc = savepath + "/" + f
    print("download:", url)
    if not os.path.exists(loc):
        req.urlretrieve(url, loc)

# GZip 압축해제
for f in files:
    gz_file = savepath + "/" + f
    raw_file = savepath + "/" + f.replace(".gz", "")
    print("gzip:", f)
    with gzip.open(gz_file, "rb") as fp:
        body = fp.read()
        with open(raw_file, "wb") as w:
            w.write(body)

print("ok")

download: http://yann.lecun.com/exdb/mnist/train-images-idx3-ubyte.gz
download: http://yann.lecun.com/exdb/mnist/train-labels-idx1-ubyte.gz
download: http://yann.lecun.com/exdb/mnist/t10k-images-idx3-ubyte.gz
download: http://yann.lecun.com/exdb/mnist/t10k-labels-idx1-ubyte.gz
gzip: train-images-idx3-ubyte.gz
gzip: train-labels-idx1-ubyte.gz
gzip: t10k-images-idx3-ubyte.gz
gzip: t10k-labels-idx1-ubyte.gz
ok


In [21]:
# 변환 프로그램

import struct

def to_csv(name, maxdata):
    
    # 레이블 파일과 이미지 파일 열기
    lbl_f = open("./mnist/"+name+"-labels-idx1-ubyte", "rb")
    img_f = open("./mnist/"+name+"-images-idx3-ubyte", "rb")
    csv_f = open("./mnist/"+name+".csv", "w", encoding="utf-8")
    
    # 헤더 정보 읽기
    mag, lbl_count = struct.unpack(">II", lbl_f.read(8))
    mag, img_count = struct.unpack(">II", img_f.read(8))
    rows, cols = struct.unpack(">II", img_f.read(8))
    pixels = rows * cols
    
    # 이미지 데이터를 읽고 csv로 저장하기
    res = []
    for idx in range(lbl_count):
        if idx > maxdata: break
        label = struct.unpack("B", lbl_f.read(1))[0]
        bdata = img_f.read(pixels)
        sdata = list(map(lambda n: str(n), bdata))
        csv_f.write(str(label)+",")
        csv_f.write(",".join(sdata)+"\r\n")
        
        # 잘 저장됐는지 이미지 파일로 저장해서 테스트하기
        if idx < 10:
            s = "P2 28 28 255\n"
            s += " ".join(sdata)
            iname = "./mnist/{0}-{1}-{2}.pgm".format(name, idx, label)
            with open(iname, "w", encoding="utf-8") as f:
                f.write(s)
    csv_f.close()
    lbl_f.close()
    img_f.close()

to_csv("train", 1000)
to_csv("t10k", 500)

In [22]:
# 이미지 데이터 학습시키기

from sklearn import model_selection, svm, metrics

# csv 파일을 읽어 들이고 가공하기
def load_csv(fname):
    labels = []
    images = []
    with open(fname, "r") as f:
        for line in f:
            cols = line.split(",")
            if len(cols) < 2: continue
            labels.append(int(cols.pop(0)))
            vals = list(map(lambda n: int(n) / 256, cols))
            images.append(vals)
    return {"labels":labels, "images":images}

data = load_csv("./mnist/train.csv")
test = load_csv("./mnist/t10k.csv")
# 학습하기
clf = svm.SVC()
clf.fit(data["images"], data["labels"])
# 예측하기
predict = clf.predict(test["images"])
# 결과 확인하기
ac_score = metrics.accuracy_score(test["labels"], predict)
cl_report = metrics.classification_report(test["labels"], predict)
print("정답률 = ", ac_score)
print("리포트 = ")
print(cl_report)

정답률 =  0.7884231536926147
리포트 = 
             precision    recall  f1-score   support

          0       0.87      0.93      0.90        42
          1       0.81      1.00      0.89        67
          2       0.84      0.69      0.76        55
          3       0.87      0.57      0.68        46
          4       0.76      0.75      0.75        55
          5       0.63      0.80      0.71        50
          6       0.97      0.67      0.79        43
          7       0.74      0.86      0.79        49
          8       0.91      0.72      0.81        40
          9       0.71      0.81      0.76        54

avg / total       0.80      0.79      0.79       501



## SVM ? 

SVM : Support Vector Machine

SVM은 선을 구성하는 매개변수를 조정해서 요소들을 구분하는 선을 찾고, 이를 기반으로 패턴을 인식하는 방법

A와 B라는 두가지 패턴을 구분하는 방법을 찾는 것이 패턴 인식의 목표

A와 B라는 패턴을 벡터로 나타내서 평면 위에 올리고 구분선을 그리게 되는데 이 때 패턴의 경계가 되는 것을 식별평면이라한다.

식별평면에서 패턴들과의 거리(마진)를 최대로 만드는 것이 가장 좋은 결과이다.

이는 SVM의 특징인 "마진 최대화" 방침.

SVM을 이용하면 알 수 없는 패턴도 제대로 분류할 확률이 높다. : 일반화 능력(범화 능력)